# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-1. Применяем простейший локальный поиск.

В этой задаче Вам предлагается попробовать стандартную технику локального поиска (local search) в применении к задаче о сбалансированном разбиении графа. Мы будем рассматривать безвесовый вариант задачи с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* [Скачайте](http://mat.gsia.cmu.edu/COLOR/instances.html#XXMYC) файлы mycielX.col  (`for X in range(1,8)`).  (Если интересно, откуда такие графы берутся и чем интересны, см. конструкцию Зыкова—Мыцельского [здесь](https://docs.com/dainiak/3327).)
* Для каждого из графов найдите локальным поиском локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Ваша функция `basic_local_search` должна принимать на вход граф в формате, предоставляемом функцией `read_col_file`, и возвращать найденное локально минимальное разбиение просто как множество либо список вершин, лежащих в одной любой из двух компонент разбиения.

In [3]:
def read_col_file(filename: str) -> tuple:
    with open(filename, 'r') as file:
        vertices, edges = set(), set()
        for line in file:
            line = line.strip()
            if line.startswith('p'):
                vertices = set(range(1, int(line.split()[-2]) + 1))
            elif line.startswith('e'):
                edges.add(tuple(map(int, line.split()[-2:])))
        return (vertices, edges)

In [125]:
import random

def cut_cost(L: set, R: set, edges:set) -> int:
        ans = 0
        for edge in edges:
            v1, v2 = edge
            if (v1 in L and v2 in R) or (v1 in R and v2 in L):
                ans += 1
        return ans

def random_partition(vertices):
    L = set(random.sample(vertices, len(vertices) // 2))
    R = vertices - L
    
    return (L, R)

def basic_local_search(graph: tuple) -> tuple:
    
    vertices, edges = graph
    n = len(vertices)
    L, R = random_partition(vertices)
    cur_cut_cost = cut_cost(L, R, edges)
    
    while True:
        min_new_L = []
        min_new_R = []
        min_new_cut_cost = cur_cut_cost

        # перебираем 2 вершины, которые мы поменяем местами
        for v1 in L:
            for v2 in R:
                new_L = L.copy()
                new_R = R.copy()
                
                new_L.remove(v1)
                new_R.remove(v2)
                new_L.add(v2)
                new_R.add(v1)
                
                cur_cut_cost = cut_cost(new_L, new_R, edges)
                if cur_cut_cost < min_new_cut_cost:
                    min_new_L = new_L
                    min_new_R = new_R
                    min_new_cut_cost = cur_cut_cost
        
        if min_new_L == []:
            return (L, R)
        else:
            L = min_new_L
            R = min_new_R
            if (len(L) > len(R)):
                L, R = R, L
            cur_cut_cost = min_new_cut_cost

In [79]:
# Запустите эту ячейку, чтобы скачать файлы с графами
# Если все сломалось, пробуйте, пока не получится

import urllib.request

url_template = 'http://mat.gsia.cmu.edu/COLOR/instances/myciel{num}.col'


for i in range(3, 8):
    with open("myciel{num}.col".format(num=i), "w") as file:
        url = url_template.format(num=i)
        response = urllib.request.urlopen(url)
        print(response.read().decode('utf-8'), file=file)


In [128]:
%%time
for i in range(3, 8):
    print("Myciel{num}:".format(num=i))
    
    vertices, edges = read_col_file('myciel{num}.col'.format(num=i))
    L, R = basic_local_search((vertices, edges))
    cut = cut_cost(L, R, edges)
    
    print(L, '\n', R, '\n', "cut cost = ", cut, '\n')

Myciel3:
{4, 6, 9, 10, 11} 
 {1, 2, 3, 5, 7, 8} 
 cut cost =  8 

Myciel4:
{2, 5, 8, 11, 14, 17, 18, 19, 20, 21, 23} 
 {1, 3, 4, 6, 7, 9, 10, 12, 13, 15, 16, 22} 
 cut cost =  28 

Myciel5:
{1, 4, 5, 6, 8, 9, 10, 11, 15, 17, 19, 20, 21, 25, 27, 28, 34, 36, 38, 39, 40, 43, 45} 
 {2, 3, 7, 12, 13, 14, 16, 18, 22, 23, 24, 26, 29, 30, 31, 32, 33, 35, 37, 41, 42, 44, 46, 47} 
 cut cost =  95 

Myciel6:
{1, 4, 9, 11, 12, 17, 18, 19, 20, 21, 23, 24, 34, 35, 39, 40, 41, 43, 44, 47, 51, 54, 56, 58, 64, 65, 67, 68, 70, 71, 72, 74, 75, 77, 79, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 93, 95} 
 {2, 3, 5, 6, 7, 8, 10, 13, 14, 15, 16, 22, 25, 26, 27, 28, 29, 30, 31, 32, 33, 36, 37, 38, 42, 45, 46, 48, 49, 50, 52, 53, 55, 57, 59, 60, 61, 62, 63, 66, 69, 73, 76, 78, 80, 84, 92, 94} 
 cut cost =  293 

Myciel7:
{2, 5, 6, 7, 8, 10, 13, 23, 25, 28, 29, 30, 31, 33, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 50, 53, 55, 60, 61, 62, 64, 66, 67, 69, 71, 72, 73, 74, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87,